<h4 align = center> Dr. Jesús Martí Gavilá. Departamento de Ingeniería Cartográfica, Geodesia y fotogrametría </h4>
<font color="blue" ><h1 align=center>PRÁCTICA 8</h1> <p> <h1 align=center> CURSO BÁSICO PYTHON: FOLIUM</h1></font>

<a href="http://www.upv.es/titulaciones/GTI/indexc.html"><img src = "http://www.upv.es/imagenes/marcaUPVN1.png" width =200 ></a>

<h2 align=center>Grado Tecnologías Interactivas | Tecnologías de la Información Geográfica</h2>



<div class="alert alert-block alert-info" style="margin-top: 20px">
<b>FOLIUM</b> es una biblioteca de visualización de Python que se desarrolló con el único fin de visualizar datos geoespaciales. Es una biblioteca completamente gratuita. <p>  
<h2>Tabla de Contenido</h2>   
    <ul>
        <li>
            <a>Acerca de los conjuntos de datos (DataFrame)</a>
        </li>
        <li>
            <a>Mapa Puntual</a>
        </li>
        <li>
            <a>Mapa de calor</a>
        </li>
        <li>
            <a>Mapa de calor con función tiempo</a>
        </li>
        <li>
            <a>Tareas a realizar</a>
        </li>
    </ul>
<p>
<b> Base de datos: </b> <a href="https://datos.madrid.es/portal/site/egob"> https://datos.madrid.es/portal/site/egob </a>
<p>
<b> Documentación: </b> <a href="https://python-visualization.github.io/folium/"> https://python-visualization.github.io/folium/</a><p>    

</div>


<H3> Alumnos: </H3> Tarea hecha por parejas, los dos hemos hecho la Tarea 1 y la Tarea 2, Somos: Carles Mart

<h1 align=center><font color="green" >Estudio de la accidentalidad de la ciudad de Madrid mediante mapa de calor y puntual.</font></h1><p>
    
<h2>Caso de estudio</h2>
Crear un mapa de calor y puntual de los accidentes originados por vehiculos y bicicletas en el año 2021

In [ ]:
# Librerias necesarias
import openpyxl
import pandas as pd
import numpy as np
import folium
from folium import plugins
import webbrowser
from pyproj import Transformer

In [ ]:
# Base de datos
base_datos= pd.read_excel('2021_Accidentalidad.xlsx')
base_datos.head(5)

# Preparación Base de datos

In [ ]:
print(base_datos.dtypes) # imprime el tipo de campo
print("Numero de registros =" , len( base_datos)) # imprime el tamaño de la base de datos

In [ ]:
# Elimina los registros en blanco en los campos de coordenadas
accidentes_df = base_datos.dropna(subset=['coordenada_x_utm', 'coordenada_y_utm'])

# Transformación de coordenadas

La geolocalización en el fichero de accidentes del ayuntamiento de Madrid se realiza con coordenadas UTM ETRS89 en el Huso 30. Para representar coordenadas en FOLIUM se necesitan coordenadas geográficas en el mismo sistema

In [ ]:

from pyproj import Transformer

# Establecer los sistemas de transformación epsg:xxxx de entrada y epsg:yyyyy de salida
transformacion = Transformer.from_crs('epsg:xxxxx','epsg:yyyyy',always_xy=True) 
puntos = list(zip(accidentes_df.coordenada_x_utm,accidentes_df.coordenada_y_utm)) # Crea una lista con todos los puntos

coorgeo = np.array(list(transformacion.itransform(puntos)))
accidentes_df.loc[:,'longitud'] = coorgeo[:,0]
accidentes_df.loc[:,'latitud'] = coorgeo[:,1]

accidentes_df.head()

# Selección de registros base de datos

In [ ]:
print(accidentes_df["tipo_persona"].unique())

In [ ]:
# Crear una nueva Dataframe que cumple las condiciones establecidas
acci_df=accidentes_df.loc[(accidentes_df['tipo_persona']=='Conductor')] 
                          
# si quisieramos introduccir más condicionantes --- & (accidentes_df['tipo_vehiculo']=='Turismo')& (accidentes_df['rango_edad']=='De 18 a 20 años')]

<h2><font color="blue" >Mapa Puntual (MarkerCluster)</font></h2><p>

Los mapas puntuales permiten plasmar variables georeferenciadas, para un mejor entendimiento por parte del usuario de la distribucción de los mismas.<p>
    
<b> Diferentes plugins de Folium: </b> <a href="https://python-visualization.github.io/folium/plugins.html"> https://python-visualization.github.io/folium/plugins.html </a>    
<p>

<b> Iconos con prefijo 'fa': </b> <a href="https://fontawesome.com/v5.15/icons?d=gallery&p=2&m=free"> https://fontawesome.com/v5.15/icons?d=gallery&p=2&m=free</a>    

In [ ]:
print(accidentes_df["tipo_vehiculo"].unique())

In [ ]:
# Crear el mapa base donde representar
mapa_accidentes = folium.Map(location=(40.43,-3.65), tiles = 'OpenStreetMap', zoom_start = 12)

# Crear diferentes dataframes 
coche_df=acci_df.loc[(acci_df['tipo_vehiculo']=='Turismo')] 
bici_df=acci_df.loc[(acci_df['tipo_vehiculo']=='Bicicleta')]

# crear un objeto de grupo de marcas para los incidentes en el DataFrame
coches = plugins.MarkerCluster( name="Accidentes_coche",).add_to(mapa_accidentes)
bicis = plugins.MarkerCluster( name="Accidentes_bicis",).add_to(mapa_accidentes)

# procesar el DataFrame y agregar cada punto de datos al grupo de marcas creado anteriormente
for lat, lng, label, in zip(coche_df.latitud, coche_df.longitud, coche_df['num_expediente']): #dos formas diferentes de llamar a un campo
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="car", prefix = 'fa'),
        popup=label,
        ).add_to(coches)

for lat, lng, label, in zip(bici_df.latitud, bici_df.longitud, bici_df['num_expediente']): #dos formas diferentes de llamar a un campo
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="blue", icon="bicycle", prefix = 'fa'),
        popup=label,
        ).add_to(bicis)

#Añadir controles
folium.LayerControl().add_to(mapa_accidentes)
draw = plugins.Draw(export=True)
draw.add_to(mapa_accidentes)

#Muestra mapa
mapa_accidentes

 <h1 align=center><font color="green" >Ubicación de contenedores vidrio y envases en Madrid. Mapa puntual.</font></h1><p> 

In [ ]:
'''
La base de datos es Contenedores_varios.csv, utilizamos ahora un fichero CSV
Es opcional colocar el tipo de separador y la codificación. Por defecto es , y UTF-8 peró hay ficheros csv
con formato sep=';', encoding = 'cp1252')
'''
contenedores_df = pd.read_csv('Contenedores_varios.csv', sep=';',low_memory = False)
contenedores_df.head()

In [ ]:
print(contenedores_df["Tipo Contenedor"].unique())

In [ ]:
vidrio_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='VIDRIO')] 
envases_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ENVASES')] 

# mapa dual
mapa_contenedores = folium.Map(location=(40.43,-3.65), tiles = 'OpenStreetMap', zoom_start = 12)

# map tiles
folium.TileLayer('OpenStreetMap').add_to(mapa_contenedores)

vidrio = plugins.MarkerCluster( name="Contenedores Vidrio").add_to(mapa_contenedores)
envases = plugins.MarkerCluster( name = 'Contenedores envases').add_to(mapa_contenedores)

for lat, lng, label in zip(vidrio_df.LATITUD, vidrio_df.LONGITUD, vidrio_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(vidrio)

for lat, lng, label in zip(envases_df.LATITUD, envases_df.LONGITUD, envases_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="info-sign"),
        popup=label,
        ).add_to(envases)

# Añadir control de mapas
folium.LayerControl().add_to(mapa_contenedores)

# Crear mapas
mapa_contenedores.save("madrid_contenedores.html")
webbrowser.open_new_tab('madrid_contenedores.html')

<h2><font color="blue" >Mapa de Calor (HeatMap)</font></h2><p>

Los mapas de calor, o heatmaps, son un tipo de representación dentro de de la simbología de mapas que nos permite mostrar al usuario los puntos calientes o conjuntos de datos más relevantes dentro de una nube de puntos. Con una combinación de opciones de simbología y datos cuantitativos provenientes de base de datos georeferenciadas, obtendras mapas que reflejan concentraciones o distribuciones espaciales de variables no continuas en el espacio.

In [ ]:
# Utilizaremos el pluging HeatMap
# la lista de datos puede ser de 2 formas (Latitud, Longitud) o (Latitud, Longitud y Peso).

mapa_calor = folium.Map(location=(40.43,-3.65), tiles = 'cartodb positron', zoom_start = 12)
plugins.HeatMap(data=acci_df[['latitud', 'longitud']], radius=15, use_local_extrema=True, name="Mapa de calor").add_to(mapa_calor)
folium.LayerControl().add_to(mapa_calor)
mapa_calor

In [ ]:
# publicar el mapa
mapa_calor.save("heatmap_acci.html")
webbrowser.open_new_tab('heatmap_acci.html') #Abre el HTML en una ventana del navegador

 <h2><font color="blue" >Mapa de calor con evolución en el tiempo (HeatMapWithTime) </font></h2><p>
Este plugin nos permite generar mapas de calor de diferentes periodos de tiempo especificados y representarlos mediante automatizacion con un play que nos permite represesentar la evolución del fenómeno

In [ ]:
# Crear una nueva Dataframe que cumpla unas determinadas condiciones
acci_df=accidentes_df.loc[(accidentes_df['tipo_persona']=='Conductor') & (accidentes_df['tipo_vehiculo']=='Turismo')]

In [ ]:
# Crear un campo en el Dataframe con el valor del mes extraido de la fecha
import datetime
mes = pd.DatetimeIndex(acci_df['fecha']).month 
acci_df = acci_df.assign(mes = mes)  

In [ ]:
# Generar lista para el slider del visualizador
lista_tiempo = ['Enero','Febrero','Marzo','Abril','Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre','Diciembre']

In [ ]:
lista_peso = [] # Crear una lista vacia
acci_df = acci_df.assign(peso = 1)  # crear un campo nuevo con valor por defecto 1

# Rellenar la lista con los campos (Latitud, Longitud y Peso) necesarios para HeatMap
for x in acci_df['mes'].sort_values().unique():
   lista_peso.append(acci_df.loc[acci_df['mes']==x,   # Agrupar los valores con latitud y longitud igual 
                  ['latitud','longitud','peso']].groupby(['latitud','longitud']) 
                  .sum().reset_index().values.tolist())

In [ ]:
print(lista_peso     )

In [ ]:
# Generar el mapa
madrid_map = folium.Map(location=(40.43,-3.65), control_scale = True, tiles = "cartodb positron", zoom_start = 12)

# Ejecutar el pluging HeatMapWithTime
plugins.HeatMapWithTime(lista_peso, radius = 30, index = lista_tiempo,
                        auto_play = False, min_opacity = 0.5, max_opacity = 1, use_local_extrema=True, name="Accidentes_coche_2020").add_to(madrid_map)

# publica el mapa
folium.LayerControl().add_to(madrid_map)
madrid_map.save("madrid_acci.html")
webbrowser.open_new_tab('madrid_acci.html')

<h2><font color="blue" >Mapa Puntual </font></h2><p>
 Este plugin nos permite en una misma ventana Html tener 2 mapas
 <p>

 <h1 align=center><font color="green" >Ubicación de contenedores vidrio y envases en Madrid.Mapa puntual </font></h1><p>   

In [ ]:
'''
La base de datos es Contenedores_varios.csv, utilizamos ahora un fichero CSV
Es opcional colocar el tipo de separador y la codificación. Por defecto es , y UTF-8 peró hay ficheros csv
con formato sep=';', encoding = 'cp1252')
'''
contenedores_df = pd.read_csv('Contenedores_varios.csv', sep=';',low_memory = False)
contenedores_df.head()

In [ ]:
print(contenedores_df["Tipo Contenedor"].unique())

In [ ]:
vidrio_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='VIDRIO')] 
envases_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ENVASES')] 

# mapa dual
mapa_dual = plugins.DualMap(location=[40.4, -3.65], tiles=None, zoom_start=20)

# map tiles
folium.TileLayer('OpenStreetMap').add_to(mapa_dual.m1)
folium.TileLayer('CartoDB Positron').add_to(mapa_dual.m2)
vidrio = plugins.MarkerCluster( name="Contenedores Vidrio").add_to(mapa_dual.m1)
envases = plugins.MarkerCluster( name = 'Contenedores envases').add_to(mapa_dual.m2)

for lat, lng, label in zip(vidrio_df.LATITUD, vidrio_df.LONGITUD, vidrio_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(vidrio)

for lat, lng, label in zip(envases_df.LATITUD, envases_df.LONGITUD, envases_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="info-sign"),
        popup=label,
        ).add_to(envases)

# Añadir control de mapas
folium.LayerControl().add_to(mapa_dual)

# Crear mapas
mapa_dual.save("madrid_dual.html")
webbrowser.open_new_tab('madrid_dual.html')

<h2><font color="red" >TAREA 1</font></h2><p>
<h3> Realizar una webMapping de residuos de la ciudad de Madrid.</h3> <p>
Crear capas diferenciadas de mapa de calor de los residuos (vidrio, envases) y un mapa único con la ubicación de los contenedores diferenciados por el tipo de contenedor (vidrio, envases, papel, organico).<P>
    <h4>Guardar como: contenedores_"apellido alumno".html</h4>

In [ ]:
# Librerias necesarias
import openpyxl
import pandas as pd
import numpy as np
import folium
from folium import plugins
import webbrowser
from pyproj import Transformer

In [ ]:
# Abrir la base de datos Contenedores_varios.csv
# Base de datos
contenedores_df = pd.read_csv('Contenedores_varios.csv', sep=';',low_memory = False)
contenedores_df.head()
print(contenedores_df["Tipo Contenedor"].unique())

In [ ]:
# Crear diferentes dataframes para los residuos indicados
vidrio_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='VIDRIO')] 
envases_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ENVASES')] 
papel_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='PAPEL-CARTON')] 
organico_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ORGANICA')] 

# mapa puntual
mapa_contenedores = folium.Map(location=(40.43,-3.65), tiles = 'OpenStreetMap', zoom_start = 12)

# map tiles
folium.TileLayer('OpenStreetMap').add_to(mapa_contenedores)

vidrio = plugins.MarkerCluster( name="Contenedores Vidrio").add_to(mapa_contenedores)
envases = plugins.MarkerCluster( name = 'Contenedores envases').add_to(mapa_contenedores)
papel = plugins.MarkerCluster( name="Contenedores papel").add_to(mapa_contenedores)
organico = plugins.MarkerCluster( name = 'Contenedores organico').add_to(mapa_contenedores)

for lat, lng, label in zip(vidrio_df.LATITUD, vidrio_df.LONGITUD, vidrio_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(vidrio)

for lat, lng, label in zip(envases_df.LATITUD, envases_df.LONGITUD, envases_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="info-sign"),
        popup=label,
        ).add_to(envases)
for lat, lng, label in zip(papel_df.LATITUD, papel_df.LONGITUD, papel_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="blue", icon="info-sign"),
        popup=label,
        ).add_to(papel)

for lat, lng, label in zip(organico_df.LATITUD, organico_df.LONGITUD, organico_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="grey", icon="info-sign"),
        popup=label,
        ).add_to(organico)

    
plugins.HeatMap(data=vidrio_df[['LATITUD', 'LONGITUD']], radius=15, use_local_extrema=True, name="Mapa de calor Vidrio").add_to(mapa_contenedores)
plugins.HeatMap(data=envases_df[['LATITUD', 'LONGITUD']], radius=15, use_local_extrema=True, name="Mapa de calor Envases").add_to(mapa_contenedores)



# Añadir control de mapas
folium.LayerControl().add_to(mapa_contenedores)
mapa_contenedores.save("Contenedores_Marti_Femenia_Y_Pastor_Gonzalez.html")
webbrowser.open_new_tab('Contenedores_Marti_Femenia_Y_Pastor_Gonzalez.html')


<h2><font color="red" >TAREA 2</font></h2><p>
<h3> Realizar webmapping con mapas de calor de accidentes en 2022</h3><p>
Realizar varios mapas de calor de accidentes producidos en la ciudad de Madrid, a lo largo del 2022 y distribuido por meses, que se han visto involucrados conductores diferenciados por sexo (Elegir: Hombre o Mujer) y con vehiculos distintos al turismo.

In [ ]:
# Cargar y limpiar tabla de blancos
# Base de datos
base_datos= pd.read_excel('2022_Accidentalidad.xlsx')
base_datos.head(5)

print(base_datos.dtypes) # imprime el tipo de campo
print("Numero de registros =" , len( base_datos)) # imprime el tamaño de la base de datos
# Elimina los registros en blanco en los campos de coordenadas
accidentes_df = base_datos.dropna(subset=['coordenada_x_utm', 'coordenada_y_utm'])

In [ ]:
# Transformar coordenadas
from pyproj import Transformer

# Establecer los sistemas de transformación epsg:xxxx de entrada y epsg:yyyyy de salida
transformacion = Transformer.from_crs('epsg:25830','epsg:4258',always_xy=True) 
puntos = list(zip(accidentes_df.coordenada_x_utm,accidentes_df.coordenada_y_utm)) # Crea una lista con todos los puntos

coorgeo = np.array(list(transformacion.itransform(puntos)))
accidentes_df.loc[:,'longitud'] = coorgeo[:,0]
accidentes_df.loc[:,'latitud'] = coorgeo[:,1]

accidentes_df.head()

In [ ]:
# Crear una nueva Dataframe que cumpla unas determinadas condiciones
print(accidentes_df["tipo_persona"].unique())
# Crear una nueva Dataframe que cumple las condiciones establecidas
acci_df=accidentes_df.loc[(accidentes_df['sexo']=='Hombre')&(accidentes_df['tipo_vehiculo']!='Turismo')] 

In [ ]:

# Crear un campo en el Dataframe con el valor del mes extraido de la fecha
import datetime
mes = pd.DatetimeIndex(acci_df['fecha']).month 
acci_df = acci_df.assign(mes = mes)  

In [ ]:
# lista para el slider del visualizador
lista_tiempo = ['Enero','Febrero','Marzo','Abril','Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre','Diciembre']

In [ ]:
# Crear una lista vacia
lista_peso = [] # Crear una lista vacia
acci_df = acci_df.assign(peso = 1)  # crear un campo nuevo con valor por defecto 1

# Rellenar la lista con los campos (Latitud, Longitud y Peso) necesarios para HeatMap
for x in acci_df['mes'].sort_values().unique():
   lista_peso.append(acci_df.loc[acci_df['mes']==x,   # Agrupar los valores con latitud y longitud igual 
                  ['latitud','longitud','peso']].groupby(['latitud','longitud']) 
                  .sum().reset_index().values.tolist())
# crear un campo nuevo con valor por defecto 1

# Rellenar la lista con los campos (Latitud, Longitud y Peso) necesarios para HeatMap


In [ ]:
# Generar el mapa

# Ejecutar el pluging HeatMapWithTime

# publica el mapa. Accidentes_Madrid_"apellido alumno".html
# Generar el mapa
madrid_map = folium.Map(location=(40.43,-3.65), control_scale = True, tiles = "cartodb positron", zoom_start = 12)

# Ejecutar el pluging HeatMapWithTime
plugins.HeatMapWithTime(lista_peso, radius = 30, index = lista_tiempo,
                        auto_play = False, min_opacity = 0.5, max_opacity = 1, use_local_extrema=True, name="Accidentes_coche_2020").add_to(madrid_map)

# publica el mapa
folium.LayerControl().add_to(madrid_map)
madrid_map.save("Madrid_Marti_Femenia_Y_Pastor_Gonzalez.html")
webbrowser.open_new_tab('Madrid_Marti_Femenia_Y_Pastor_Gonzalez.html')
